In [47]:
import os
import numpy as np
import nanonispy2 as nap
import yaml
from datetime import datetime
from PIL import Image

def get_scan(file_name, crop_unfinished: bool = True):
    if not os.path.exists(file_name):
        print(f"Error: File \"{file_name}\" does not exist.")
        return
    else:
        scan_data = nap.read.Scan(file_name) # Read the scan data. scan_data is an object whose attributes contain all the data of the scan
        channels = np.array(list(scan_data.signals.keys())) # Read the various channels
        scan_header = scan_data.header
        up_or_down = scan_header.get("scan_dir", "down") # Read whether the scan was recorded in the upward or downward direction
        
        # Stack the forward and backward scans for each channel in a big tensor. Flip the backward scan
        scan_tensor_uncropped = np.stack([np.stack((np.array(scan_data.signals[channel]["forward"], dtype = float), np.flip(np.array(scan_data.signals[channel]["backward"], dtype = float), axis = 1))) for channel in channels])
        if up_or_down == "up": scan_tensor_uncropped = np.flip(scan_tensor_uncropped, axis = 2) # Flip the scan if it recorded in the upward direction

        # Determine which rows should be cropped off in case the scan was not completed
        pixels_uncropped = scan_header.get("scan_pixels", np.array([100, 100], dtype = int)) # Read the old number of pixels in the scan
        scan_range_uncropped = scan_header.get("scan_range", np.array([1E-8, 1E-8], dtype = float)) # Read the old size of the scan
        masked_array = np.isnan(scan_tensor_uncropped[0, 1]) # All channels have the same number of NaN values. The backward scan has more NaN values because the scan always starts in the forward direction.
        nan_counts = np.array([sum([int(masked_array[j, i]) for i in range(len(masked_array))]) for j in range(len(masked_array[0]))])
        good_rows = np.where(nan_counts == 0)[0]
        scan_tensor = np.array([[scan_tensor_uncropped[channel, 0, good_rows], scan_tensor_uncropped[channel, 1, good_rows]] for channel in range(len(channels))])
        
        pixels = np.shape(scan_tensor[0, 0]) # The number of pixels is recalculated on the basis of the scans potentially being cropped
        scan_range = np.array([scan_range_uncropped[0], scan_range_uncropped[1] * pixels[1] / pixels_uncropped[1]]) # Recalculate the size of the slow scan direction after cropping
        scan_range_nm = [round(scan_dimension, 3) for scan_dimension in scan_range * 1E9] # Return the scan range in nanometer
        
        # Get the bias
        bias = round(float(scan_header.get("bias", 0)), 3)

        # Extract and convert z-controller parameters
        z_controller = scan_header.get("z-controller")
        feedback = bool(z_controller.get("on")[0])
        setpoint_str = z_controller.get("Setpoint")[0]
        
        # Extract and convert time parameters
        rec_date = [int(element) for element in scan_data.header.get("rec_date", "00.00.1900").split(".")]
        rec_time = [int(element) for element in scan_data.header.get("rec_time", "00:00:00").split(":")]
        dt_object = datetime(rec_date[2], rec_date[1], rec_date[0], rec_time[0], rec_time[1], rec_time[2])

        # Re-unitize
        scan_range_nm = [round(dimension * 1E9, 3) for dimension in scan_range] # Scan range in nanometer
        setpoint_pA = round(float(setpoint_str.split()[0]) * 1E12, 3) # Setpoint in pA
        scan_tensor_nm_pA = scan_tensor
        for index in range(len(channels)):
            channel = channels[index]
            if channel == "X" or channel == "Y" or channel == "Z":
                scan_tensor_nm_pA[index, 0] *= 1E9
                scan_tensor_nm_pA[index, 1] *= 1E9
            elif channel == "Current":
                scan_tensor_nm_pA[index, 0] *= 1E12
                scan_tensor_nm_pA[index, 1] *= 1E12

        # Add new attributes to the scan object
        setattr(scan_data, "bias", bias)
        setattr(scan_data, "channels", channels)
        setattr(scan_data, "scan_tensor_uncropped", scan_tensor_uncropped) # Uncropped means the size of the scan before deleting the rows that were not recorded
        setattr(scan_data, "pixels_uncropped", pixels_uncropped)
        setattr(scan_data, "scan_range_uncropped", scan_range_uncropped)
        setattr(scan_data, "scan_tensor", scan_tensor)
        setattr(scan_data, "scan_tensor_nm_pA", scan_tensor_nm_pA)
        setattr(scan_data, "pixels", pixels)
        setattr(scan_data, "scan_range", scan_range)
        setattr(scan_data, "scan_range_nm", scan_range_nm)
        setattr(scan_data, "feedback", feedback)
        setattr(scan_data, "setpoint_pA", setpoint_pA)
        setattr(scan_data, "date_time", dt_object)

        return scan_data

scan_object = get_scan("C:\\Nanonis\\simulation0008.sxm")

In [ ]:
pil_img = Image.fromarray(scan_object.scan_tensor_nm_pA[0, 0])
scan_tensor_nm_pA

In [ ]:
with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "r") as f:
    config = yaml.safe_load(f)

print(config)

with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "w") as f:
    yaml.safe_dump({"last_folder": "Blabla"}, f)


{'last_folder': 'Blabla'}


In [14]:
try: # Save the scan folder to the config yaml file so it opens automatically on startup next time
    with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "w") as file:
        yaml.safe_dump({"last_file": "C:\\Scripts\\Scanalyzer\\scanalyzer\\dummy_scan.sxm"}, file)
except Exception as error:
    print("Failed to save the scan folder to the config.yml file.")
    print(error)

In [8]:
try: # Read the last scan file from the config yaml file
    with open("C:\\Scripts\\Scanalyzer\\scanalyzer\\config.yml", "r") as file:
        config = yaml.safe_load(file)
        last_file = config.get("last_file")
except:
    print("Failed to load the last scan folder from the config.yml file.")

print(last_file)

C:\Scripts\Scanalyzer\scanalyzer\dummy_scan.sxm
